## Integrated FM Control

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from dataclasses import dataclass, field

microscope, settings = utils.setup_session()

In [ ]:
# Controls
# Light Source 
    # Power
# Filter Wheel
    # Excitation Wavelength
    # Emission Wavelength
# Objective Lens
    # Magnification
    # Position
    # Move Relative
    # Move Absolute
# Camera
    # Exposure Time
    # Binning
    # Gain
    # Offset

# Tasks
# Acquire Image
# Acquire Z-Stack
# Acquire Overview
# Move to Position
# Save OME Metadata
# Load OME Metadata

# TODO:
# - tests
# 


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from pprint import pprint

from fibsem.fm.microscope import FluorescenceMicroscope
from fibsem.fm.structures import ChannelSettings, ZParameters
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus

# create microscope 
fm = FluorescenceMicroscope()
# OME-standard -> Static instrument configuration vs dynamic image acquisition settings

zparams = ZParameters(zmin=-10e-6, zmax=10e-6, zstep=1e-6)

ch1 = ChannelSettings(
    name="Channel-01", 
    excitation_wavelength=488.0, 
    emission_wavelength=520.0, 
    power=100.0, 
    exposure_time=200.0, 
)
ch2 = ChannelSettings(
    name="Channel-02", 
    excitation_wavelength=561.0, 
    emission_wavelength=None, 
    power=80.0, 
    exposure_time=150.0,
)



In [ ]:
# acquire a single image
image = fm.acquire_image(ch1)
print(image.data.shape, image.data.dtype)
print(f"Acquired image with shape: {image.data.shape} and dtype: {image.data.dtype}")
pprint(image.metadata)
plt.imshow(image.data, cmap='gray')
plt.title(f"Acquired Image - {ch1.name}")
plt.colorbar()
plt.show()



In [ ]:
# Acquire images for multiple channels
images = acquire_channels(fm, [ch1, ch2])
for idx, image in enumerate(images.data):
    print(f"Image {idx+1} Metadata:")
    md = images.metadata[idx]
    pprint(md)
    print(f"Image {idx+1} Shape: {image.shape}, Dtype: {image.dtype}")
    # Optionally display the image
    plt.imshow(image[0].data, cmap='gray')
    plt.title(f"Acquired Image - {md['objective']['magnification']}x")
    plt.colorbar()
    plt.show()



In [ ]:
# Acquire Z-Stack

fm.objective.move_absolute(0.0)  # Move objective to the starting position
image = acquire_z_stack(fm, ch1, zparams)
print(image.data.shape)
md = image.metadata

img: np.ndarray

nc, nz, ny, nx = image.data.shape

for idx in range(nc):
    for zidx in range(nz):
        img = image.data[idx, zidx, :, :]
        print(f"Z-Stack Image {zidx} Metadata:")
        # pprint(md[idx])
        print(f"Z-Stack Image {zidx} Shape: {img.shape}, Dtype: {img.dtype}")
        # Optionally display the image
        plt.imshow(img, cmap='gray')
        obj_pos = md[idx]["objective-positions"][zidx]
        plt.title(f"Z-Stack Image - Z Position: {obj_pos:.2e} microns")
        plt.colorbar()
        plt.show()


In [ ]:
# Run autofocus
fm.objective.move_absolute(0.0)  # Reset objective position to 0
best_focus = run_auto_focus(fm, ch1)
print(f"Best focus position: {best_focus:.2e} microns")

## Odemis FM Control

In [ ]:
%load_ext autoreload
%autoreload 2

import fibsem.fm.odemis as fm_odemis
from odemis import model

focuser = fm_odemis.model.getComponent(role="focus")

print(focuser.position.value)
print(focuser.getMetadata())


In [ ]:
objective = fm_odemis.OdemisObjectiveLens(None)

print(objective.position)
print(objective.magnification)
print(objective.numerical_aperture)

In [ ]:
objective.insert()
print(objective.position)
objective.retract()
print(objective.position)

In [ ]:
odemis_light_source = model.getComponent(role="light")
print(odemis_light_source.power.value)
print("SPECTRA")
print(odemis_light_source.spectra.value) # 5D (99% low, 25% low, centre, 25%high 99% high) x wavelengths

In [ ]:
fm = fm_odemis.OdemisFluorescenceMicroscope(None)

# from fibsem.fm.microscope import FluorescenceMicroscope
# fm = FluorescenceMicroscope()

# set filter wheel
fm.filter_set.emission_wavelength = 488
print(fm.filter_set.emission_wavelength)

# reflection mode
fm.filter_set.emission_wavelength = None
print(fm.filter_set.emission_wavelength)

In [ ]:
from fibsem.fm.structures import ChannelSettings
from pprint import pprint

fm.objective.insert()

ch = ChannelSettings(name="channel-01", 
                     excitation_wavelength=405, 
                     emission_wavelength=None, 
                     exposure_time=0.5, 
                     power=0.5, # W 
                    )

# fm.set_binning(2)
# fm.set_power(0.3)

image = fm.acquire_image(ch)

pprint(image.data.shape)
pprint(image.metadata)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from pprint import pprint
plt.imshow(image.data, cmap="gray")
plt.show()


print(image.data.shape)
pprint(image.metadata)
pprint(fm.get_metadata())

In [ ]:
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus
from fibsem.fm.structures import ZParameters
zparams = ZParameters(zmin=-5e-6, zmax=5e-6, zstep=1e-6)

fm.objective.insert()
image = acquire_z_stack(fm, ch, zparams)

print(image.data.shape)
pprint(image.metadata)


In [ ]:
ch1 = ChannelSettings(name="channel-01", 
                     excitation_wavelength=405, 
                     emission_wavelength=None, 
                     exposure_time=0.5, 
                     power=0.5, # W 
                     )

ch2 = ChannelSettings(name="channel-02",
                     excitation_wavelength=488, 
                     emission_wavelength=500,
                     exposure_time=0.1, 
                     power=0.1, # W 
                     )

multi_channel_settings = [ch1, ch2]
channel_image = acquire_channels(fm, multi_channel_settings)

print(channel_image.data.shape)
for idx, img in enumerate(channel_image.data):
    print(f"Channel {idx+1} Image Shape: {img.shape}, Dtype: {img.dtype}")
    plt.imshow(img[0], cmap='gray')
    plt.title(f"Channel {idx+1} Image")
    plt.colorbar()
    plt.show()

filename = "test-single-channel.ome.tiff"
channel_image.save(filename)

from ome_types import from_tiff

ome_metadata = from_tiff(filename)
print(ome_metadata.to_xml())



In [ ]:
# Acquire Z-Stack with multiple channels
image = acquire_z_stack(fm, multi_channel_settings, zparams)

print(image.data.shape)
pprint(image.metadata)

In [ ]:

from fibsem.fm.structures import FluorescenceImage
import tifffile as tff
import numpy as np
from ome_types import to_xml, from_tiff

filename = "image.ome.tiff"
image.save(filename)

ome = from_tiff(filename)
print(to_xml(ome))
tff.OmeXml.validate(to_xml(ome))  # Validate the OME XML from the TIFF file

with tff.TiffReader(filename) as tif:

    arr = tif.asarray()
    print(arr.shape)  # Should be (C, Z, Y, X) for a multi-channel Z-stack
    print(tif.ome_metadata)  # OME metadata in XML format

for idx, img in enumerate(image.data):
    print(f"Z-Stack Image {idx+1} Metadata:")
    pprint(image.metadata[idx])
    print(f"Z-Stack Image {idx+1} Shape: {img.shape}, Dtype: {img.dtype}")
    # Optionally display the image
    plt.imshow(np.amax(img, axis=0), cmap='gray')
    plt.title(f"Z-Stack Image - Z Position: {image.metadata[idx]['objective']['position']:.2e} microns")
    plt.colorbar()
    plt.show()


## Live Acquisition

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import time
from fibsem.fm.structures import FluorescenceImage
from fibsem.fm.microscope import FluorescenceMicroscope
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.DEBUG)

fm = FluorescenceMicroscope()

# from fibsem.fm.odemis import OdemisFluorescenceMicroscope
# fm = OdemisFluorescenceMicroscope(None)

# from fibsem.fm.thermo_fisher import ThermoFisherFluorescenceMicroscope
# fm = ThermoFisherFluorescenceMicroscope(None)

fm.set_exposure_time(0.5)

def on_acquisition_signal(image: FluorescenceImage):
    print("Acquisition signal received!")
    print(f"Image shape: {image.data.shape}")
    print(f"Acquisition Date: {image.metadata.acquisition_date}")
    print(f"Channel Name: {image.metadata.channels[0].name}")
    print(f"Excitation Power: {image.metadata.channels[0].power} W")
    print(f"Exposure Time: {image.metadata.channels[0].exposure_time} s")
    print(f"Excitation Wavelength: {image.metadata.channels[0].excitation_wavelength} nm")
    print(f"Emission Wavelength: {image.metadata.channels[0].emission_wavelength} nm")
    print(f"Objective Magnification: {image.metadata.channels[0].objective_magnification}x")
    print(f"Objective Position: {image.metadata.channels[0].objective_position:.2e} m")
    print(f"Image Data Type: {image.data.dtype}")
    print("-"*80)
    # plt.imshow(image.data, cmap='gray')
    # plt.show()

def on_acquisition_signal_second(image: FluorescenceImage):
    print("Second acquisition signal received!")
    print(f"-"*80)


fm.acquisition_signal.disconnect(on_acquisition_signal)
# fm.acquisition_signal.disconnect(on_acquisition_signal_second)
fm.acquisition_signal.connect(on_acquisition_signal)
# fm.acquisition_signal.connect(on_acquisition_signal_second)


In [ ]:
# start the acquisition
fm.start_acquisition()

time.sleep(5)

# stop acquisition
fm.stop_acquisition()

## FIBSEM Control

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils
# from fibsem.fm.odemis import OdemisFluorescenceMicroscope
from fibsem.fm.microscope import FluorescenceMicroscope
from fibsem.fm.structures import ChannelSettings, ZParameters, FluorescenceImage
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus
from fibsem.structures import FibsemStagePosition
import tifffile as tff
from pprint import pprint

microscope, settings = utils.setup_session()

# fm = OdemisFluorescenceMicroscope(microscope)

In [ ]:
print(f"Emission Wavelengths: {microscope.fm.filter_set.available_emission_wavelengths}")
print(f"Excitation Wavelengths: {microscope.fm.filter_set.available_excitation_wavelengths}")

In [ ]:
stage_position = microscope.get_stage_position()
print(f"Stage Position: {stage_position}")

In [ ]:
stage_positions = [
    FibsemStagePosition(x=0, y=0, z=0, r=0, t=0, coordinate_system="RAW"),
    FibsemStagePosition(x=50e-6, y=0, z=0, r=0, t=0, coordinate_system="RAW"),
    FibsemStagePosition(x=0, y=50e-6, z=0, r=0, t=0, coordinate_system="RAW"),
]

# acquire parameters
channel_settings = [
    ChannelSettings(
    name="channel-01",
    excitation_wavelength=488,
    emission_wavelength=None,
    power=0.5,
    exposure_time=0.1,
),
ChannelSettings(
    name="channel-02",
    excitation_wavelength=561,
    emission_wavelength=600,
    power=0.5,
    exposure_time=0.1,
)]
zparams = ZParameters(zmin=-5e-6, zmax=5e-6, zstep=500e-9)
objective_position = -2.2e-3  # Example objective position in meters

for stage_position in stage_positions:

    # move to each position
    microscope.move_stage_absolute(stage_position)
    print(f"Moved to stage position: {stage_position}")

    # move objective to position
    microscope.fm.objective.move_absolute(objective_position)
    print(f"Moved objective to position: {microscope.fm.objective.position} m")

    # acquire z-stack
    image = acquire_z_stack(microscope.fm, 
                            channel_settings=channel_settings, 
                            zparams=zparams)

    # print(image.metadata)
    filename = f"z-stack-{stage_position.x:.2e}-{stage_position.y:.2e}.ome.tiff"
    image.save(filename)

    img2 = FluorescenceImage.load(filename)
    pprint(img2.metadata.to_dict())

    print(f"-"*80)

In [ ]:
%matplotlib inline
from fibsem.structures import ImageSettings, BeamType
import matplotlib.pyplot as plt

idx = 0
settings.image.beam_type = BeamType.ION

while True:
    # acquire image
    fm_image = microscope.fm.acquire_image()

    print(f"Acquired image {idx+1} with shape: {fm_image.data.shape} and dtype: {fm_image.data.dtype}")

    fib_image = microscope.acquire_image(settings.image)
    print(f"Acquired fibsem image {idx+1} with shape: {fib_image.data.shape} and dtype: {fib_image.data.dtype}")

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    ax[0].imshow(fm_image.data, cmap='gray')
    ax[0].set_title(f"Fluorescence Image {idx+1}")
    ax[1].imshow(fib_image.data, cmap='gray')
    ax[1].set_title(f"Fibsem Image {idx+1}")
    plt.show()

    idx += 1

    if idx >= 5:  # Limit to 5 images for demonstration
        break

In [ ]:
microscope.move_stage_absolute(FibsemStagePosition(x=0, y=0, z=0, r=0, t=0, coordinate_system="RAW"))
microscope.move_flat_to_beam(BeamType.ION)

In [ ]:
print(microscope.get_stage_position())
microscope.move_to_microscope("FM")
print(microscope.get_stage_orientation())
print(microscope.get_stage_position())

In [ ]:
print(microscope.get_stage_position())

In [ ]:
print(microscope.get_stage_position())
microscope.move_to_microscope("FIBSEM")
print(microscope.get_stage_orientation())
print(microscope.get_stage_position())


In [ ]:
# Acquire Z-Stack with multiple channels
image = acquire_z_stack(microscope.fm, channel_settings, zparams)
image.save("multi-channel-z-stack.ome.tiff")

## Coincidence Milling


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils
from fibsem.milling import FibsemMillingStage
from fibsem.structures import Point
from fibsem.milling.base import FibsemMillingSettings
from fibsem.milling.core import setup_milling, draw_patterns, run_milling, finish_milling
from fibsem.milling.patterning.patterns2 import RectanglePattern
from fibsem.fm.microscope import FluorescenceMicroscope, FluorescenceImage
from fibsem.fm.structures import ChannelSettings, ZParameters
import time
microscope, settings = utils.setup_session()


In [ ]:
milling_stage= FibsemMillingStage(name="Milling Stage",
                                  milling=FibsemMillingSettings(milling_current=60e-12),
                                  pattern=RectanglePattern(width=10e-6, 
                                                           height=5e-6, 
                                                           depth=1e-6, 
                                                           point=Point(x=0, y=5e-6)))
milling_stage.alignment.enabled = False

channel_settings = ChannelSettings(
    name="Channel-01",
    excitation_wavelength=488.0,
    emission_wavelength=None,
    power=0.001,
    exposure_time=0.1,
)

# setup milling
setup_milling(microscope, milling_stage)
draw_patterns(microscope, patterns=milling_stage.pattern.define())


def on_fm_acquisition_signal(image: FluorescenceImage):
    print("FM Acquisition Signal Received!")
    print(f"Image shape: {image.data.shape}")
    print(f"Acquisition Date: {image.metadata.acquisition_date}")
    print(f"Channel Metadata: {image.metadata.channels[0].to_dict()}")
    print(f"Milling State: {microscope.get_milling_state()}")
    print("-"*80)

# stop milling if it is running
microscope.stop_milling()

# connect the acquisition signal
microscope.fm.acquisition_signal.disconnect()
microscope.fm.acquisition_signal.connect(on_fm_acquisition_signal)

# start fm
microscope.fm.start_acquisition(channel_settings=channel_settings)

microscope.start_milling()
time.sleep(5)  # Wait for some time to allow acquisition

# stop fm
microscope.stop_milling()
microscope.fm.stop_acquisition()
print("Milling finished. FM acquisition stopped.")

# finalize
microscope.fm.acquisition_signal.disconnect(on_fm_acquisition_signal)
finish_milling(microscope)

#### Overview Acquisition


In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from fibsem.fm.acquisition import acquire_tileset, ChannelSettings
from fibsem.structures import BeamType

from fibsem.fm.acquisition import stitch_tileset
import matplotlib.pyplot as plt
from pprint import pprint


microscope, settings = utils.setup_session()


In [ ]:

channel_settings = ChannelSettings(
    name="Channel-01",
    excitation_wavelength=488.0,
    emission_wavelength=None,
    power=0.1,
    exposure_time=0.5,
)


# To acquire a tileset on arctis:
# pre-tilt = 0
# orientation=FM
# beam_type=BeamType.ELECTRON

microscope.system.stage.shuttle_pre_tilt = 0
microscope.move_flat_to_beam(beam_type=BeamType.ELECTRON)
print(microscope.get_stage_position())
images = acquire_tileset(microscope=microscope, 
                         channel_settings=channel_settings,
                         grid_size=(3, 3), 
                         tile_overlap=0.0, 
                         beam_type=BeamType.ELECTRON)

# To acquire a tileset on meteor/iflm:
# pre-tilt = 35
# orientation=FIB (FM)
# beam_type=BeamType.ION
# project stage position to sample-stage coordinates

# microscope.system.stage.shuttle_pre_tilt = 35
# microscope.move_flat_to_beam(beam_type=BeamType.ION)
# print(microscope.get_stage_position())
# images = acquire_tileset(microscope=microscope, 
#                          channel_settings=channel_settings,
#                          grid_size=(3, 3), 
#                          tile_overlap=0.0, 
#                          beam_type=BeamType.ION)



print(microscope.get_stage_position())

In [ ]:

stitched_image = stitch_tileset(images, tile_overlap=0.1,)

plt.imshow(stitched_image.data, cmap='gray')
plt.title("Stitched Image from Tileset")
plt.colorbar()
plt.show()

pprint(stitched_image.metadata.to_dict())

In [ ]:

xs, ys = [], []
x1, y1 = [], []
z1 = []
for i in range(len(images)):
    for j in range(len(images[i])):
        # print(f"Image {i}, Channel {j}: Shape: {images[i][j].data.shape}, Dtype: {images[i][j].data.dtype}")
        # plt.imshow(images[i][j].data[0, 0], cmap='gray')
        # plt.title(f"Tile {i}, Channel {j}")
        # plt.colorbar()
        # plt.show()

        img = images[i][j]
        # for FM-Arctis
        stage_position1 = img.metadata.stage_position
        x1.append(stage_position1.x)
        y1.append(stage_position1.y)
        z1.append(stage_position1.z)
        # for FM-Meteor/iFLM
        # stage_position = microscope.project_stable_move(dx=0, dy=0,
        #                                                 beam_type=BeamType.ION, 
        #                                                 base_position=img.metadata.stage_position)

        # print(stage_position1)
        # print(stage_position)

        # xs.append(stage_position.x)
        # ys.append(stage_position.y)

        # pprint(img.metadata.to_dict())


# TODO: i need a version of stable move with BeamType.PHOTON

# To stitch from position, we need to project the stage positions to the sample-stage coordinates
# requires: pre-tilt, orientation, beam_type
plt.figure(figsize=(8, 6))
plt.scatter(xs, ys, c='blue', marker='o')
plt.scatter(x1, y1, c='red', marker='x', label='Original Stage Position')
plt.title("Tile Positions")
plt.xlabel("X Position (m)")
plt.ylabel("Y Position (m)")
plt.grid()
plt.axis('equal')
plt.show()

# plot x, z

plt.figure(figsize=(8, 6))
plt.scatter(x1, z1, c='red', marker='x', label='Original Stage Position')
plt.title("Tile Positions (X vs Z)")
plt.xlabel("X Position (m)")
plt.ylabel("Z Position (m)")
plt.grid()
plt.axis('equal')
plt.show()

# plot y, z
plt.figure(figsize=(8, 6))
plt.scatter(y1, z1, c='red', marker='x', label='Original Stage Position')
plt.title("Tile Positions (Y vs Z)")
plt.xlabel("Y Position (m)")
plt.ylabel("Z Position (m)")
plt.grid()
plt.axis('equal')
plt.show()


In [ ]:
# calculate angle between yz
import numpy as np

def calculate_angle_between_yz(y, z):
    """
    Calculate the angle between the Y and Z coordinates.
    
    Args:
        y (float): Y coordinate.
        z (float): Z coordinate.
        
    Returns:
        float: Angle in degrees.
    """
    angle = np.arctan2(z, y)  # Calculate angle in radians
    return np.degrees(angle)  # Convert to degrees


angles = [calculate_angle_between_yz(y, z) for y, z in zip(y1, z1)]
print("Angles between Y and Z coordinates:")
for idx, angle in enumerate(angles):
    print(f"Tile {idx+1}: {angle:.2f} degrees")

#### Coincidence Milling Strategy


In [ ]:
%load_ext autoreload
%autoreload 2

import time
from dataclasses import dataclass, field
from fibsem import utils
from fibsem.milling import FibsemMillingStage
from fibsem.structures import Point
from fibsem.milling.base import FibsemMillingSettings
from fibsem.milling.core import setup_milling, draw_patterns, run_milling, finish_milling
from fibsem.milling.patterning.patterns2 import RectanglePattern
from fibsem.fm.microscope import FluorescenceMicroscope, FluorescenceImage
from fibsem.fm.structures import ChannelSettings, ZParameters
from fibsem.microscope import FibsemMicroscope
from fibsem.milling.base import MillingStrategy, MillingStrategyConfig


microscope, settings = utils.setup_session()


In [ ]:
milling_stage= FibsemMillingStage(name="Milling Stage",
                                  milling=FibsemMillingSettings(milling_current=60e-12),
                                  pattern=RectanglePattern(width=10e-6, 
                                                           height=5e-6, 
                                                           depth=1e-6, 
                                                           point=Point(x=0, y=5e-6)))
milling_stage.alignment.enabled = False

channel_settings = ChannelSettings(
    name="Channel-01",
    excitation_wavelength=488.0,
    emission_wavelength=None,
    power=0.001,
    exposure_time=0.1,
)


@dataclass
class CoincidenceMillingStrategyConfig(MillingStrategyConfig):
    """Configuration for the Coincidence Milling Strategy."""
    pass


class CoincidenceMillingStrategy(MillingStrategy):
    """Coincidence Milling Strategy for milling and fluorescence acquisition."""
    name: str = "CoincidenceMilling"
    fullname: str = "fibsem.milling.CoincidenceMillingStrategy"
    config_class = CoincidenceMillingStrategyConfig

    def run(self, microscope: FibsemMicroscope, stage: FibsemMillingStage, asynch: bool = False, parent_ui = None) -> None:
        """Co"""

        # setup milling
        setup_milling(microscope, stage)
        draw_patterns(microscope, patterns=stage.pattern.define())

        # stop milling if it is running
        microscope.stop_milling()

        # connect the acquisition signal
        microscope.fm.acquisition_signal.disconnect()
        microscope.fm.acquisition_signal.connect(self.on_fm_acquisition_signal)

        # start fm
        microscope.fm.start_acquisition(channel_settings=channel_settings)

        microscope.start_milling()
        time.sleep(5)  # Wait for some time to allow acquisition

        # stop fm
        microscope.stop_milling()
        microscope.fm.stop_acquisition()
        print("Milling finished. FM acquisition stopped.")

        # finalize
        microscope.fm.acquisition_signal.disconnect()
        finish_milling(microscope)

    def on_fm_acquisition_signal(self, image: FluorescenceImage):
        print("FM Acquisition Signal Received!")
        print(f"Image shape: {image.data.shape}")
        print(f"Acquisition Date: {image.metadata.acquisition_date}")
        # print(f"Channel Metadata: {image.metadata.channels[0].to_dict()}")
        print(f"Milling State: {microscope.get_milling_state()}")
        # TODO: handle feedback here...

        # calc mean intensity
        mean_intensity = image.data.mean()
        print(f"Mean Intensity: {mean_intensity:.2f}")
        print("-"*80)

In [ ]:

milling_stage.strategy = CoincidenceMillingStrategy()
milling_stage.run(microscope)